# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

city_weather_df = pd.read_csv("../output_data/Retrieved_City_Weather_Data.csv")
city_weather_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Ancud,-41.8697,-73.8203,64.42,70,99,4.41,CL,1639837461
1,Bilibino,68.0546,166.4372,-22.25,94,100,2.64,RU,1639837461
2,Moose Factory,51.2585,-80.6094,-5.80,77,1,4.61,CA,1639837462
3,Cabo San Lucas,22.8909,-109.9124,65.95,80,24,5.44,MX,1639837462
4,Bengkulu,-3.8004,102.2655,75.06,96,100,4.63,ID,1639837462
...,...,...,...,...,...,...,...,...,...
580,Port Lincoln,-34.7333,135.8667,62.06,78,9,2.37,AU,1639837628
581,Kariba,-16.5167,28.8000,74.80,79,99,2.04,ZW,1639837628
582,Seryshevo,51.0933,128.3786,-18.83,96,77,5.70,RU,1639837492
583,Anchorage,61.2181,-149.9003,25.65,90,90,5.01,US,1639837628


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [4]:
#Use the Lat and Lng as locations and Humidity as the weight
locations_df = city_weather_df[["Lat", "Lng"]].astype(float)
locations_df



,Lat,Lng
0,-41.8697,-73.8203
1,68.0546,166.4372
2,51.2585,-80.6094
3,22.8909,-109.9124
4,-3.8004,102.2655
...,...,...
580,-34.7333,135.8667
581,-16.5167,28.8000
582,51.0933,128.3786
583,61.2181,-149.9003


In [5]:
#Add Heatmap layer to map
humidity_df = city_weather_df["Humidity"].astype(float)
humidity_df


0      70.0
1      94.0
2      77.0
3      80.0
4      96.0
       ... 
580    78.0
581    79.0
582    96.0
583    90.0
584    77.0
Name: Humidity, Length: 585, dtype: float64

In [6]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, 
                                 dissipating=False,max_intensity=100,point_radius = 3)
    
   

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
#Ideal_weather_df=city_weather_df.dropna()
Ideal_weather_df=city_weather_df.copy()

#A max temperature lower than 80 degrees but higher than 70.
Ideal_weather_df = Ideal_weather_df.loc[(Ideal_weather_df["Max Temp"] < 80) & (Ideal_weather_df["Max Temp"] > 70)]


#Wind speed less than 10 mph.
Ideal_weather_df = Ideal_weather_df.loc[(Ideal_weather_df["Wind Speed"] < 10)]
Ideal_weather_df
                                       
# Zero cloudiness.
Ideal_weather_df = Ideal_weather_df.loc[(Ideal_weather_df["Clouds"]==0)]
Ideal_weather_df

  # Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Ideal_weather_df=Ideal_weather_df.dropna()
Ideal_weather_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
188,Mar del Plata,-38.0023,-57.5575,77.02,57,0,6.91,AR,1639837518
216,Pisco,-13.7000,-76.2167,71.02,78,0,5.75,PE,1639837525
514,Rabat,33.9911,-6.8401,73.04,64,0,4.61,MA,1639837608
576,Molina,-35.1167,-71.2833,75.36,61,0,3.44,CL,1639837627


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#Store into variable named hotel_df.
hotel_df=Ideal_weather_df

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
188,Mar del Plata,-38.0023,-57.5575,77.02,57,0,6.91,AR,1639837518,
216,Pisco,-13.7000,-76.2167,71.02,78,0,5.75,PE,1639837525,
514,Rabat,33.9911,-6.8401,73.04,64,0,4.61,MA,1639837608,
576,Molina,-35.1167,-71.2833,75.36,61,0,3.44,CL,1639837627,


In [9]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    #get hotel based on location, lat and long
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #extract resukts
    results = response['results']
    
    #Save those name into a dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Cannot find the result")
        print("-------------------------")
       
    


Retrieving Results for Index 188: Mar del Plata.
Closest hotel in Mar del Plata is NH Gran Hotel Provincial.
Retrieving Results for Index 216: Pisco.
Closest hotel in Pisco is Hotel San Jorge Residencial.
Retrieving Results for Index 514: Rabat.
Closest hotel in Rabat is The View Hotel Rabat.
Retrieving Results for Index 576: Molina.
Closest hotel in Molina is Lagar Hotel Boutique..


In [11]:
#print the data frame
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
188,Mar del Plata,-38.0023,-57.5575,77.02,57,0,6.91,AR,1639837518,NH Gran Hotel Provincial
216,Pisco,-13.7000,-76.2167,71.02,78,0,5.75,PE,1639837525,Hotel San Jorge Residencial
514,Rabat,33.9911,-6.8401,73.04,64,0,4.61,MA,1639837608,The View Hotel Rabat
576,Molina,-35.1167,-71.2833,75.36,61,0,3.44,CL,1639837627,Lagar Hotel Boutique.


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))